In [22]:
import tensorflow as tf

In [23]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [24]:
input_width=28
input_height=28
input_channels=1
input_pixels=28*28

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10
input_data_shape_to_hidden_layer=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [25]:
weights={"wc1":tf.Variable(tf.random_normal((conv1_k, conv1_k, input_channels, n_conv1))),
        "wc2":tf.Variable(tf.random_normal((conv2_k, conv2_k, n_conv1, n_conv2))),
        "wh1":tf.Variable(tf.random_normal((input_data_shape_to_hidden_layer, n_hidden))),
        "wo":tf.Variable(tf.random_normal((n_hidden, n_out)))}
biases={"bc1":tf.Variable(tf.random_normal((n_conv1, ))),
       "bc2":tf.Variable(tf.random_normal((n_conv2, ))),
       "h1":tf.Variable(tf.random_normal((n_hidden, ))),
       "o":tf.Variable(tf.random_normal((n_out, )))}

In [26]:
def conv(x, weights, bias, strides=1):
    out=tf.nn.conv2d(x, weights, padding='SAME', strides=[1, strides, strides, 1])
    out=tf.nn.bias_add(out, bias)
    out=tf.nn.relu(out)
    return out
def maxpooling(x, k=2):
    out=tf.nn.max_pool(x, padding='SAME', ksize=[1, k, k, 1], strides=[1, k, k, 1])
    return out

In [27]:
def cnn(x, weights, biases, keep_prob):
    x=tf.reshape(x, shape=(-1, input_width, input_height, input_channels))
    
    conv1=conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool=maxpooling(conv1, max_pool1_k)
    
    conv2=conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool=maxpooling(conv2, max_pool2_k)
    
    hidden_input=tf.reshape(conv2_pool, shape=(-1, input_data_shape_to_hidden_layer))
    hidden_output_before_activation=tf.add(tf.matmul(hidden_input, weights['wh1']), biases['h1'])
    hidden_output_before_dropout=tf.nn.relu(hidden_output_before_activation)
    hidden_output=tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output=tf.add(tf.matmul(hidden_output, weights['wo']), biases['o'])
    
    return output

In [28]:
x=tf.placeholder('float', shape=(None, input_pixels))
y=tf.placeholder(tf.int32, shape=(None, n_out))
keep_prob=tf.placeholder('float')
pred=cnn(x, weights, biases, keep_prob)

In [29]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

In [30]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [31]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [32]:
batch_size=100
for i in range(25):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x, batch_y=mnist.train.next_batch(batch_size)
        c, _=sess.run([cost, optimize], feed_dict={x:batch_x, y:batch_y, keep_prob:0.8})
        total_cost+=c
    print(total_cost)

1124752.728392601
35438.513766527176
20082.036641046405
12863.45922262936
10128.057869623877
7485.568730003319
5640.705014332414
5839.190242731944
5591.353065775326
5435.020169103333
4602.166464297453
3953.924775457326
3595.380793705983
3999.303467959119
3730.5781304190145
3235.418271073861
2528.513306436459
2607.5781089261927
2077.1452489381663
2428.745748791452
1903.4942767930902
1736.5551482450885
2226.557587512974
2074.130589367208
1751.3817009122104


In [33]:
predictions=tf.argmax(pred, axis=1)
correct_labels=tf.argmax(y, axis=1)
boolean_number_of_correct_predictions=tf.equal(predictions, correct_labels)
preds, correct_preds=sess.run([predictions, boolean_number_of_correct_predictions], feed_dict={x:mnist.test.images,
                                                                                               y:mnist.test.labels,
                                                                                               keep_prob:1.0})
correct_preds.sum()/len(correct_preds)

0.9838